In [3]:
import tika
import tika.parser
from tika import parser
import requests
import urllib.parse
import pandas as pd
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
#run "python -mhttp.server 8888" in the directory where your photos are

#run ifconfig in terminal to find your ip address listed for en0

images_url_usc  = 'http://10.25.179.208:8888'

images_url_usc2 = 'http://10.25.77.81:8888'

images_url_home = 'http://192.168.1.13:8888'

images_url_MOC = 'http://10.25.188.76:8888'

# Webscraping all 80,581 URLs

In [5]:
import requests
from bs4 import BeautifulSoup
import csv

webpage_url = images_url_home

response = requests.get(webpage_url)
html_content = response.content

soup = BeautifulSoup(html_content, 'html.parser')

image_urls_set = set()
count = 0
for link in soup.find_all('a'):
    href = link.get('href')
    if href is not None and href.startswith('http'):
        image_urls_set.add(href)
        count += 1
    elif href is not None:
        image_urls_set.add(webpage_url + '/' + href)
        count += 1
    print(f"Processed {count} links", end='\r')

image_urls = list(image_urls_set)

# Save the image URLs to a text file
with open('image_urls.txt', mode='w') as file:
    for url in image_urls:
        file.write(url + '\n')
        
print("All links processed!") # added for confirmation


All links processed!s


In [6]:
#verifying there were no mistakes durnig link extraction

with open('image_urls.txt', 'r') as f:
    num_links = len(f.readlines())

print(f'The file contains {num_links} links.')


with open('image_urls.txt', 'r') as f:
    unique_links = set(line.strip() for line in f if line.strip())

num_unique_links = len(unique_links)

print(f'The file contains {num_unique_links} unique links.')


The file contains 80581 links.
The file contains 80581 unique links.


# Image Captions JSON file creation utilizing threading

- `docker run -it -p 8764:8764 uscdatascience/im2txt-rest-tika`
- test_url = 'http://localhost:8764/inception/v3/caption/image?url=http://192.168.1.13:8888/Pixstory-image-164318304274781.png'

In [7]:
import threading
import queue
import time
import json

# cnt = 0

# specify the starting index for processing images
# start_index = 0

# create a queue to hold the image URLs
image_url_queue = queue.Queue()

# populate the queue with the image URLs starting from the specified index
for image_url in image_urls[0:]:
    image_url_queue.put(image_url)

# create a list to hold the processed image data
image_captions_data = []

# Set starting index
start_index = 70100

# remove the top items from the queue based 
for i in range(start_index):
    image_url_queue.queue.popleft()

# Creating file name to hold image captions data
image_captions_JSON_data = 'image_captions_data_master.json'

def process_images():

    while True:

        # get the next image URL from the queue
        image_url = image_url_queue.get()

        # construct the API URL for the image classification
        api_captions_url = 'http://localhost:8764/inception/v3/caption/image?url=' + image_url

        # send the API request
        response = requests.get(api_captions_url)

        # parse the JSON response
        json_response = response.json()
        # print(json_response)

        # extract the first listed sentence
        first_sentence = json_response['captions'][0]['sentence']

        # create a dictionary to store the image URL and its caption
        data = {'url': image_url, 'caption': first_sentence}
        # print(data)

        # add the dictionary to the list of image data
        image_captions_data.append(data)

        # save the image data to disk periodically
        if len(image_captions_data) % 100 == 0:
            with open(image_captions_JSON_data, 'w') as f:
                json.dump(image_captions_data, f)
            print(f'Saved {len(image_captions_data)} images to file {image_captions_JSON_data}.\nCompleted Chunk {int(len(image_captions_data)/100) + (int(start_index/100)-1)}.\nImage URL List Current Index: {int(len(image_captions_data)) + int(start_index)}', flush=True)

        # mark the image URL as processed
        image_url_queue.task_done()

# create a pool of worker threads to process the images
num_threads = 20
for i in range(num_threads):
    t = threading.Thread(target=process_images)
    t.daemon = True
    t.start()

# wait for all images to be processed
image_url_queue.join()

# save the final set of image data to disk
# with open('image_captions_data3.json', 'w') as f:
#     json.dump(image_captions_data, f)
# print('Saved image data to disk', flush=True)

Saved 100 images to file image_captions_data_master.json.
Completed Chunk 701.
Image URL List Current Index: 70200
Saved 200 images to file image_captions_data_master.json.
Completed Chunk 702.
Image URL List Current Index: 70300
Saved 300 images to file image_captions_data_master.json.
Completed Chunk 703.
Image URL List Current Index: 70400
Saved 400 images to file image_captions_data_master.json.
Completed Chunk 704.
Image URL List Current Index: 70500
Saved 500 images to file image_captions_data_master.json.
Completed Chunk 705.
Image URL List Current Index: 70600
Saved 600 images to file image_captions_data_master.json.
Completed Chunk 706.
Image URL List Current Index: 70700
Saved 700 images to file image_captions_data_master.json.
Completed Chunk 707.
Image URL List Current Index: 70800
Saved 800 images to file image_captions_data_master.json.
Completed Chunk 708.
Image URL List Current Index: 70900
Saved 900 images to file image_captions_data_master.json.
Completed Chunk 709.
I

KeyboardInterrupt: 

In [ ]:
# count the number of dictionaries in the list
num_image_captions = len(image_captions_data)

print(f'The JSON object contains {num_image_captions} image captions.')

The JSON object contains 118 image captions.


Processed 20 images
Processed 20 images
Processed 20 images
Processed 20 images
Processed 20 images
Processed 20 images
Processed 20 images
Processed 20 images
Processed 20 images
Processed 20 images
Saved image data to disk
Processed 20 images
Processed 20 images
Processed 20 images
Processed 20 images
Processed 20 images
Processed 20 images
Processed 20 images
Processed 20 images
Processed 20 images
Processed 20 images


In [14]:
# Count the number of JSON objects in output file

import json

filename = "image_captions_data_master.json" # replace with your file path



with open(filename) as file:
    data = json.load(file)
    count = 0
    for obj in data:
        count += len(obj)
print(f"There are {count} key-value pairs in {filename}.")



There are 20800 key-value pairs in image_captions_data_master.json.


In [16]:
filename = "image_captions_data_master.json" # replace with your file path

with open(filename) as file:
    count = 0
    for line in file:
        count += 1
print(f"There are {count} lines in {filename}.")


There are 1 lines in image_captions_data_master.json.
